In [104]:
using LinearAlgebra 
using Revise 
using TOML
using DelimitedFiles
using Pkg 
using RigidBodyDynamics
using ForwardDiff
using SparseArrays
using Rotations
using QuadrupedBalance

In [66]:
"""Maximal coorindate lqr backward ricatti""" 
function maximal_coordinate_lqr(Ad, Bd,Cd,D, Q, R, max_iter=10000)
    N = max_iter
    n = size(Q,1)
    m = size(R,1)
    l = size(Cd,2)
    P_prev = zeros(n,n)
    K_prev = zeros(m,n)
    P = zeros(n,n)
    Qn = Q
    β = 1e-5
    K = zeros(m,n)

    P_prev .= Qn
    K_prev .= K 
    for k = (N-1):-1:1
        H = [R+Bd'*P_prev*Bd     Bd'*P_prev*Cd           Bd'*D'; 
             Cd'*P_prev*Bd       β*1.0I(6)+Cd'*P_prev*Cd    Cd'*D'; 
             D*Bd                    D*Cd                        -β*1.0I(6)]

        b = [Bd'*P_prev*Ad; Cd'*P_prev*Ad; D*Ad]

        KLM = H\b
        K = KLM[1:m,:]
        L_ = KLM[m+1:m+l,:]
        M = KLM[m+l+1:end,:]

        P .= Q + K'*R*K + β*L_'*L_ + (Ad-Bd*K-Cd*L_)'*P_prev*(Ad-Bd*K-Cd*L_) - M'*D*(Ad-Bd*K-Cd*L_)
        if norm(K_prev - K) < 1e-8
            println("Backward Ricatti converged in ", k, " iterations")
            return K
        end
        P_prev[:] .= P[:]
        K_prev[:] .= K[:]
    end

    return K
end

maximal_coordinate_lqr

In [67]:
data = TOML.parsefile("ipopt_eq_point.toml") # load eq point 
x_eq = data["x_eq"];
u_eq = data["u_eq"];
λ_eq = data["λ_eq"];

In [68]:
urdfpath = joinpath(@__DIR__, "..", "src","a1","urdf","a1.urdf")
A1mech = parse_urdf(urdfpath, floating=true, remove_fixed_tree_joints=false)
A1 = QuadrupedBalance.UnitreeA1FullBody(A1mech);
foot_contacts = [1, 0, 0, 1] # FR, FL, RR, RL
foot_indices = []
for i in 1:length(foot_contacts)
    if(foot_contacts[i] == 1)
        append!(foot_indices, (i-1)*3 .+ (1:3))
    end 
end 

In [145]:
QuadrupedBalance.fk_world(x_eq)[10:12]

3-element Vector{Float64}:
 -0.18299999999999841
  0.13205000000000788
  8.798517470154366e-14

In [146]:
QuadrupedBalance.fk_world(A1, x_eq)[1:3]

3-element Vector{Float64}:
 -0.18299999999999864
  0.13205000000000808
  8.754108549169359e-14

In [151]:
QuadrupedBalance.dfk_world(x_temp)[10:12,1:19]'

19×3 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.182614   0.363707  -0.483787
 -0.115149   0.448881   0.553265
 -0.121635   0.245696  -0.308792
 -0.149519   0.41369    0.057007
  1.0        0.0        0.0
  0.0        1.0        0.0
  0.0        0.0        1.0
  0.0        0.0        0.0
  0.0        0.0        0.0
  0.0        0.0        0.0
  0.025601   0.123789   0.317322
  0.0        0.0        0.0
  0.0        0.0        0.0
  0.0        0.0        0.0
  0.0356427  0.30559   -0.121601
  0.0        0.0        0.0
  0.0        0.0        0.0
  0.0        0.0        0.0
  0.123671   0.128787  -0.090107

In [148]:
QuadrupedBalance.spatial_jacobian(A1, x_eq)[4:6, 1:19]'

19×3 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.366627     -0.263552     -0.362626
 -8.1239e-5     0.362626     -0.263552
 -0.362626     -8.1239e-5     0.366627
  0.263552     -0.366627     -8.1239e-5
  1.0           0.0           0.0
  0.0           1.0           0.0
  0.0           0.0           1.0
  0.0           0.0           0.0
  0.0           0.0           0.0
  0.000198502   0.181519     -0.0848732
  0.0           0.0           0.0
  0.0           0.0           0.0
  0.0           0.0           0.0
 -0.181436      0.000372318   0.0003644
  0.0           0.0           0.0
  0.0           0.0           0.0
  0.0           0.0           0.0
 -0.0910755     0.000360403  -0.178059
  0.0           0.0           0.0

In [152]:
QuadrupedBalance.contacts_jacobian(A1, x_temp)[1:3,:]'

18×3 adjoint(::Matrix{Float64}) with eltype Float64:
 -0.0186496   0.134035    0.329401
 -0.136635    0.345631   -0.0746327
 -0.0438457   0.0624094  -0.213573
 -0.112901   -0.922547    0.368998
  0.317534    0.318397    0.893194
 -0.941502    0.218012    0.256993
  0.0         0.0         0.0
  0.0         0.0         0.0
  0.0         0.0         0.0
  0.025601    0.123789    0.317322
  0.0         0.0         0.0
  0.0         0.0         0.0
  0.0         0.0         0.0
  0.0356427   0.30559    -0.121601
  0.0         0.0         0.0
  0.0         0.0         0.0
  0.0         0.0         0.0
  0.123671    0.128787   -0.090107

In [70]:
####### Linearizing about Eq point #########
A,B,C = QuadrupedBalance.dynamics_jacobians(A1, x_eq, u_eq, λ_eq, foot_indices)
# D_fd = QuadrupedBalance.dfk_world(x_eq)[foot_indices, :]
D_fd = QuadrupedBalance.spatial_jacobian(A1, x_eq)[foot_indices,:]
attitude_error_jacobian = blockdiag(sparse(QuadrupedBalance.quaternion_differential(x_eq[1:4])), sparse(I(33)) )
D = D_fd * attitude_error_jacobian;

In [71]:
####### Discretizing the Dynamics Matrices ####### 
n = 37 
m = 12 
n_c = 6 
h = 0.001

O = [A B C]
O = [O; zeros(m+n_c, n+m+n_c)]
O_exp = exp(O.*h)
Ad = O_exp[1:37, 1:n]
Bd = O_exp[1:37, n+1:n+m]
Cd = O_exp[1:37, n+m+1:n+m+n_c];
Ad = attitude_error_jacobian' * Ad * attitude_error_jacobian  # convert quaternion error to 3 param representation
Bd = attitude_error_jacobian' * Bd 
Cd = attitude_error_jacobian' * Cd;

In [72]:
############### Setting LQR Gains ###################
Q_gains = zeros(36); 
Q_gains[1] = (1/deg2rad(15)^2) # attitude - x
Q_gains[2] = (1/deg2rad(15)^2) # attitude - y
Q_gains[3] = (1/deg2rad(15)^2)  # attitude - z 
Q_gains[4] = (0.0)        # position - x
Q_gains[5] = (0.0)        # position - y
Q_gains[6] = (0.0)        # position - z
Q_gains[6 .+ [1,2,3,4]] .= 1 ./    ([deg2rad(15), deg2rad(15), deg2rad(15), deg2rad(15)]).^2  # Hips  (FR, FL, RR, RL)
Q_gains[6 .+ [5,6,7,8]] .= 1 ./    ([deg2rad(15), deg2rad(15), deg2rad(15), deg2rad(15)]).^2  # Thighs (FR, FL, RR, RL)
Q_gains[6 .+ [9,10,11,12]] .= 1 ./ ([deg2rad(15), deg2rad(15), deg2rad(15), deg2rad(15)]).^2   # Calves  (FR, FL, RR< RL)
Q_gains[19] = Q_gains[1] / 1000 # Angular vel - x 
Q_gains[20] = Q_gains[2] / 1000 # Angular vel - y 
Q_gains[21] = Q_gains[3] / 1000 # Angular vel - z
Q_gains[22] = Q_gains[4] / 1000 # Vel - x
Q_gains[23] = Q_gains[5] / 1000 # Vel - y
Q_gains[24] = Q_gains[6] / 1000 # Vel - z
Q_gains[25:end] .= Q_gains[7:18] / 1000; # joint damping

R_gains = zeros(12);
R_gains[[1,2,3,4]] .= 1 ./    ([4.0, 1.0, 1.0, 4.0]).^2 #  Hips (FR, FL, RR, RL)
R_gains[[5,6,7,8]] .= 1 ./    ([4.0, 1.0, 1.0, 4.0]).^2 #  Thighs
R_gains[[9,10,11,12]] .= 1 ./ ([4.0, 1.0, 1.0, 4.0]).^2 #  Calves

Q = sparse(Diagonal(Q_gains)) 
R = sparse(Diagonal(R_gains)) 
K = maximal_coordinate_lqr(Ad,Bd,Cd,D, Q,R);

Backward Ricatti converged in 7020 iterations


In [73]:
open("maximal_lqr_gain.txt", "w") do io
    writedlm(io, K)
end

In [87]:
function cay(g)
    M = 1/sqrt(1+g'g)
    [M, M*g[1], M*g[2], M*g[3]]
end

cay (generic function with 1 method)

In [121]:
function rot(q)
    Rotations.UnitQuaternion(q) * [0, 0., 1.0]
end 
g = [0.0, 0., 0.]
temp = x_eq[1:4]

ForwardDiff.jacobian(x->rot(QuadrupedBalance.L(temp)*cay(x)), g)





3×3 Matrix{Float64}:
 -0.00410813   1.99999     0.0
 -1.99999     -0.00410403  0.0
 -0.00199775  -0.00409974  0.0

In [122]:
ForwardDiff.jacobian(x->rot(x), temp) * QuadrupedBalance.get_G(temp)

3×3 Matrix{Float64}:
 -0.00410813   1.99999      1.62028e-19
 -1.99999     -0.00410403   5.22501e-19
 -0.00199775  -0.00409974  -2.8749e-19